Dependences model traing code :


Traing the model to only predict the first column:
Stage 1

In [ ]:
#!pip install datasets transformers rouge-score nltk
#!pip install evaluate

from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset
from sklearn.metrics import accuracy_score
import pandas as pd
from tqdm import tqdm



file_path = "/content/dataset_FYP.xlsx"

# Load Excel file
df = pd.read_excel(file_path, engine='openpyxl')
df=df[["Question statment","Characters","States","Start State","Transitions","Final States"]]

df.head()


df["Question statment"]=df["Question statment"].str.replace('"', ' ')
df["Characters"]=df["Characters"].str.replace('"', '').str.strip()
df["States"]=df["States"].str.replace('"', '').str.strip()
df["Start State"]=df["Start State"].str.replace('"', '').str.strip()
df["Transitions"]=df["Transitions"].str.replace('"', '').str.strip()
df["Final States"]=df["Final States"].str.replace('"', '').str.strip()



df["States"] = df["States"].str.replace(" ", "").str.replace("{", "").str.replace("}", "")
df["States"] = "{" + df["States"] + "}"
stage_1=df[["Question statment","Characters"]]

stage_1 = df[["Question statment", "Characters"]].dropna()
stage_1 = stage_1[(stage_1["Question statment"].str.strip() != "") & (stage_1["Characters"].str.strip() != "")]

stage_1 = stage_1.rename(columns={
    "Question statment": "input_text",
    "Characters": "target_text"
})


dataset1 = Dataset.from_pandas(stage_1)


tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess(example):
    model_input = tokenizer("Predict characters: " + example["input_text"],
                            truncation=True, padding="max_length", max_length=64)
    target = tokenizer(example["target_text"],
                       truncation=True, padding="max_length", max_length=16)
    model_input["labels"] = target["input_ids"]
    return model_input

# Tokenize the dataset
tokenized_dataset1 = dataset1.map(preprocess)


split = tokenized_dataset1.train_test_split(test_size=0.1,shuffle=True,seed=42)
train_dataset = split["train"]
eval_dataset = split["test"]

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]
    return {"accuracy": accuracy_score(decoded_labels, decoded_preds)}


    model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = Seq2SeqTrainingArguments(
    output_dir="./t5_stage1_model",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=30,
    weight_decay=0.01,
    save_strategy="epoch",
    logging_strategy="epoch",            # <-- Log every epoch
    logging_dir="./logs",
    save_total_limit=2,
    predict_with_generate=True,
    push_to_hub=False,
    report_to="tensorboard"              # <-- Enables TensorBoard logging
)

# Step 6: Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    compute_metrics=compute_metrics,     # <-- Include accuracy metric
)

# Step 7: Train
trainer.train()

metrics = trainer.evaluate()
print(metrics)

In [ ]:
#!unzip -q t5_stage1_model.zip -d t5_stage1_model/

Checking the stage 1 predictions

In [ ]:
"""
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load trained model and tokenizer
model_path = "/content/drive/MyDrive/T5_stage1_model"
tokenizer = T5Tokenizer.from_pretrained(model_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Set to evaluation mode
model.eval()

# Example test input (similar format to your "Question statement")
input_question = "Strings of length atleast 3"

# Prepare input for model
input_text = "Predict characters: " + input_question
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Generate prediction
output_ids = model.generate(input_ids, max_length=30, num_beams=4, early_stopping=True)

# Decode output
predicted_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Predicted Characters:", "{"+predicted_text+"}")
"""

### **Stage 2**

In [ ]:
# ===== Step 1: Load & Prepare Data =====
df = df[["Question statment", "States"]].dropna()

In [ ]:


stage_2 = df.dropna(subset=["Question statment", "States"])
stage_2 = stage_2[(stage_2["Question statment"].str.strip() != "") & (stage_2["States"].str.strip() != "")]



# ===== Step 2: Create Dataset =====
stage_2 = pd.DataFrame({
    "input_text": stage_2["Question statment"],
    "target_text": stage_2["States"]
})


dataset = Dataset.from_pandas(stage_2)

# ===== Step 3: Tokenizer & Preprocessing =====
tokenizer = T5Tokenizer.from_pretrained("t5-small")

def preprocess(example):
    model_input = tokenizer(
        "Given the DFA description, list all states in {q0,q1,...} format: " + example["input_text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )
    with tokenizer.as_target_tokenizer():
        target = tokenizer(
            example["target_text"],
            truncation=True,
            padding="max_length",
            max_length=64
        )
    model_input["labels"] = target["input_ids"]
    return model_input

tokenized_dataset = dataset.map(preprocess)

# ===== Step 4: Train-Test Split =====
split = tokenized_dataset.train_test_split(test_size=0.1, shuffle=True, seed=42)
train_dataset = split["train"]
eval_dataset = split["test"]

# ===== Step 5: Metrics =====
import numpy as np
from sklearn.metrics import accuracy_score

def compute_metrics(eval_preds):
    predictions, labels = eval_preds

    # Convert model predictions to text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in labels before decoding
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Strip spaces
    decoded_preds = [p.strip() for p in decoded_preds]
    decoded_labels = [l.strip() for l in decoded_labels]

    # Compute accuracy
    return {"accuracy": accuracy_score(decoded_labels, decoded_preds)}


# ===== Step 6: Training Setup =====
model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = Seq2SeqTrainingArguments(
    output_dir="t5_stage2_model",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=20,  # More epochs since dataset is small
    weight_decay=0.01,
    save_total_limit=1,
    predict_with_generate=True,
    logging_steps=1000,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    compute_metrics=compute_metrics
)

# ===== Step 7: Train =====
trainer.train()

In [ ]:
# ===== Step 7: Save =====
model.save_pretrained("t5_stage2_model")
tokenizer.save_pretrained("t5_stage2_model")

In [ ]:
metrics = trainer.evaluate()
print(metrics)

# **Trying the fold Method for stage 2**

In [ ]:
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import KFold
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from sklearn.metrics import accuracy_score

# ===== Load & Clean Data =====
df = pd.read_excel("/content/dataset_FYP.xlsx")  # Replace with your dataset path
df = df.dropna(subset=["Question statment", "States"])
df = df[(df["Question statment"].str.strip() != "") & (df["States"].str.strip() != "")]

# Create Hugging Face Dataset
hf_dataset = Dataset.from_pandas(pd.DataFrame({
    "input_text": df["Question statment"],
    "target_text": df["States"]
}))

# Tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Preprocessing
def preprocess(example):
    model_input = tokenizer(
        "Given the DFA description, list all states in {q0,q1,...} format: " + example["input_text"],
        truncation=True, padding="max_length", max_length=128
    )
    target = tokenizer(
        example["target_text"],
        truncation=True, padding="max_length", max_length=64
    )
    model_input["labels"] = target["input_ids"]
    return model_input

# Pre-tokenize dataset
tokenized_dataset = hf_dataset.map(preprocess)

# Accuracy function
def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    predictions = predictions.argmax(-1) if predictions.ndim == 3 else predictions
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return {"accuracy": accuracy_score([l.strip() for l in decoded_labels],
                                       [p.strip() for p in decoded_preds])}

# ===== K-Fold Cross Validation =====
kf = KFold(n_splits=5, shuffle=True, random_state=42)
fold_results = []

for fold, (train_idx, val_idx) in enumerate(kf.split(tokenized_dataset)):
    print(f"\n===== Fold {fold+1} =====")

    train_dataset = tokenized_dataset.select(train_idx)
    eval_dataset = tokenized_dataset.select(val_idx)

    model = T5ForConditionalGeneration.from_pretrained("t5-small")

    training_args = Seq2SeqTrainingArguments(
        output_dir=f"t5_stage2_fold{fold+1}",
        learning_rate=5e-5,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=10,
        weight_decay=0.01,
        predict_with_generate=True,
        logging_dir=f"./logs_fold{fold+1}",
        save_total_limit=1
    )

    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
        compute_metrics=compute_metrics
    )

    trainer.train()
    metrics = trainer.evaluate()
    print(f"Fold {fold+1} Accuracy:", metrics["eval_accuracy"])
    fold_results.append(metrics["eval_accuracy"])

# ===== Final Average Accuracy =====
print("\nAverage Accuracy:", sum(fold_results)/len(fold_results))
